In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

runcycleid = etl_helpers.start_run_cycle("dimdeliverymodes")

os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    tablename = "dimdeliverymodes"
    spark.sql(f"DROP TABLE IF EXISTS {tablename};")

    query = f"""
        CREATE TABLE {tablename}
        USING DELTA AS
            SELECT
                deliverymodeid,
                NULL AS createddate,
                NULL AS modifieddate,
                NULL AS cpal,
                NULL AS cdefault,
                NULL AS cdelete,
                name AS deliverymodename,
                CASE
                    WHEN isactive = TRUE THEN 'Y'
                ELSE
                    'N'
                END AS cactive
                FROM foi_mod.deliverymodes
                order by deliverymodeid;  
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    etl_helpers.end_run_cycle(runcycleid, 't', "dimdeliverymodes")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "dimdeliverymodes", "Credentials not available")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "dimdeliverymodes")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "dimdeliverymodes", f"An error occurred: {e}")